<a href="https://colab.research.google.com/github/WinetraubLab/coregister-xy/blob/main/coregister_xy_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://github.com/WinetraubLab/coregister-xy/blob/main/coregister_xy_2.ipynb" target="_blank">
  <img src="https://img.shields.io/badge/view%20in-GitHub-blue" alt="View in GitHub"/>
</a>

# Overview
Use this notebook to get alignment information from ImageJ image registration. Print stats for individual barcodes and calculate mapping from angled tissue slice to flat view.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# @title Notebook Inputs { display-mode: "form" }
## @markdown How to use this notebook: [See Instructions]()
import numpy as np

# @markdown Input Paths:
# @markdown Leave either path blank to load a file from local file system.
trakem_xml_path = "/content/drive/Shareddrives/Yolab - Current Projects/_Datasets/2024-09-04 Multiple Barcode Alignment/align3.xml" # @param {type:"string"}
fluorescent_patch_number = 8 # @param {type:"integer"}
# @markdown For the alignment of multiple templates to one fluorescent image, specify the patch numbers of each template in the TrakEM stack.
# template_patch_1 = 11 # @param {type:"integer"}
# template_patch_2 = 14 # @param {type:"integer"}
# template_patch_3 = 17 # @param {type:"integer"}

# @markdown Enter template patch IDs in order, separated by commas. Example: 11, 14, 17
template_patch_list = "11, 14, 17" # @param {type:"string"}

# @markdown Z-depth of each template, in um. Enter as a comma-separated list. Example: 50, 52, 54
template_z_list = "68, 52, 56" # @param {type:"string"}

# @markdown Real (x,y) locations of photobleach barcode centers, as specified by script used to photobleach. Enter in format: (1000,0), (0, 1000), (0,0)
real_centers = [0,1000], [1000, 1000], [0, 0] # @param

real_centers = np.array(real_centers)
real_centers = np.column_stack((real_centers, np.zeros(real_centers.shape[0])))

template_size = 401
um_per_pixel = 2

from google.colab import drive
from google.colab import files
# drive.mount('/content/drive/')

assert len(template_patch_list) == len(template_z_list), "Number of elements in template patch list and template z list must match"

if not trakem_xml_path:
  print("Upload saved TrakEM project:")
  uploaded = files.upload()
  trakem_xml_path = list(uploaded.keys())[0]
  trakem_xml_path = os.path.join(os.getcwd(), trakem_xml_path)


In [2]:
# @title Environment Setup
!git clone -b fmp_mapping_fx https://github.com/WinetraubLab/coregister-xy.git
%cd coregister-xy

# from plane.fit_plane import FitPlane
from plane.parse_xml import ParseXML
from plane.fit_multi_plane import FitMultiPlane
# from plane.fit_plane import FitPlane
import matplotlib.pyplot as plt
import os
from google.colab import files
import math

%cd ..

fatal: destination path 'coregister-xy' already exists and is not an empty directory.
/content/coregister-xy
/content


In [10]:
zs = [int(x) for x in template_z_list.split(",")]
templates = [int(x) for x in template_patch_list.split(",")]
fps = []
for i in range(0, len(templates)):
    fp = ParseXML.from_imagej_xml(trakem_xml_path, fluorescent_patch_number, templates[i], zs[i], None, True)
    fps.append(fp)

multi_plane = FitMultiPlane.from_aligned_fitplanes(fps, real_centers, template_size, um_per_pixel)

In [11]:
multi_plane.print_single_plane_stats()

Stats for individual barcodes:

      Template ID Patch Number  Z (um) Center (x) Center (y)  Rotation (deg)  \
0             1.0         11.0   68.00     698.93    2920.88            7.28   
1             2.0         14.0   52.00    1658.59    3113.83            7.32   
2             3.0         17.0   56.00     738.31    1995.97            6.21   
Mean                             58.67                                  6.94   
StDev                             8.33                                  0.63   

       Scaling  Shear magnitude  Shear vector (x)  Shear vector (y)  
0         1.96            -0.05             -0.99             -0.10  
1         1.87             0.03              0.72              0.70  
2         1.92             0.03              0.79             -0.62  
Mean      1.92             0.00              0.17             -0.01  
StDev     0.04             0.05              1.01              0.66  


In [19]:
print("Fitplane barcodes, in px:")
c = np.array([(project.tx + multi_plane.template_size/2, project.ty + multi_plane.template_size/2) for project in multi_plane.fitplanes])
print(c, "\n")

multi_plane.calc_fitplane_centers()

print("Fitplane barcodes, in um:")
print(multi_plane.fitplane_centers)
print()

u,v,h = multi_plane.fit_mapping_to_xy()

print("Transformation coefficients:")
print("Vector u:", u)
print("Vector v:", v)
print("Vector h:", h)

Fitplane barcodes, in px:
[[ 698.93235169 2920.87604794]
 [1658.59427735 3113.82599019]
 [ 738.31430888 1995.96596334]] 

Fitplane barcodes, in um:
[[ 729.56326015 3048.88426882   68.        ]
 [1731.28264177 3250.290297     52.        ]
 [ 770.67114278 2083.43973755   56.        ]]

Transformation coefficients:
Vector u: [ 9.89809814e-01 -2.06488987e-01  7.82808345e-14]
Vector v: [ 4.21453376e-02  1.02700015e+00 -3.11291641e-14]
Vector h: [-8.50625132e+02 -1.98055781e+03  1.00000000e+00]


In [24]:
def compute_avg_error(a, b):
    assert a.shape == b.shape, "Input arrays must have the same shape"
    if a.shape[1] == 3:
        a = a[:, :2]
        b = b[:, :2]
    return np.mean(np.linalg.norm(a - b, axis=1))

projected = []
for p in multi_plane.fitplane_centers:
    projected.append(multi_plane.get_xyz_from_uv(p))
projected = np.array(projected)
err = compute_avg_error(projected[:,:-1], multi_plane.real_centers[:,:-1])
print("Average in-plane (xy) error: ", err)

Average in-plane (xy) error:  6.968115316478089e-11


clearly outline:
1. center points Cu, Cv in pixels
2. physical position in Cx,Cy,Cz in um or mm
3. output vector u,v,h

4. (Cx Cy Cz) - ( Cu*u+ Cv*v + h ) error
- out of plane error
- error vector: parallel/perpendicular components

Goal: to use this to lay a plane in an empty 3d volume

0. individual function merges
1. in class fmp: function to map arbitrary point
2. see coregister xz fit plane functions + tester